# Preparation stuff

## Connect to Drive

In [ ]:
connect_to_drive = False

In [ ]:
#Run command and authorize by popup --> other window
if connect_to_drive:
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Install packages

In [ ]:
if connect_to_drive:
    #Install FS code
    !pip install git+https://github.com/siciliano-diag/data_utils.git
    !pip install git+https://github.com/siciliano-diag/exp_utils.git
    !pip install git+https://github.com/siciliano-diag/torch_utils.git

## IMPORTS

In [ ]:
#Put all imports here
import os
import sys

# useful ready-to-use imports
# import torch
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

## Define paths

In [ ]:
#every path should start from the project folder:
project_folder = "../"
if connect_to_drive:
    project_folder = "/content/gdrive/Shareddrives/<SharedDriveName>" #Name of Shared Drive folder
    #project_folder = "/content/gdrive/MyDrive/<MyDriveName>" #Name of MyDrive folder

#Config folder should contain hyperparameters configurations
cfg_folder = os.path.join(project_folder,"cfg")

#Data folder should contain raw and preprocessed data
data_folder = os.path.join(project_folder,"data")
raw_data_folder = os.path.join(data_folder,"raw")
processed_data_folder = os.path.join(data_folder,"processed")

#Source folder should contain all the (essential) source code
source_folder = os.path.join(project_folder,"src")
#Code can be tried in notebooks, then moved into the src folder to be imported in different notebooks

#The out folder should contain all outputs: models, results, plots, etc.
out_folder = os.path.join(project_folder,"out")

## Import own code

In [ ]:
#To import from src:

#attach the source folder to the start of sys.path
sys.path.insert(0, project_folder)
print(sys.path) # view the path and verify

#import from src directory
#from src import utils

os.chdir(source_folder)

import data_utils, exp_utils, torch_utils

['/content/gdrive/Shareddrives/Generative_math/', '/content/gdrive/Shareddrives/Generative_math/', '/content/gdrive/Shareddrives/Generative_math/src', '/content/gdrive/Shareddrives/Generative_math/src', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


# MAIN

In [ ]:
cfg = exp_utils.cfg.load_configuration(config_path=cfg_folder)

In [ ]:
cfg["data"]["data_folder"] = data_folder

In [ ]:
data = data_utils.data.load_data(**cfg["data"]) #, split_vars=custom_split_vars)

In [ ]:
loaders = torch_utils.preparation.prepare_data_loaders(data, cfg["model"]["loader_params"])

In [ ]:
cfg["model"]["in_channels"] = data["train_x"].shape[1]
cfg["model"]["out_features"] = data["train_y"].shape[1]

main_module = torch_utils.model.get_torchvision_model(**cfg["model.cnn"])

In [ ]:
callbacks = torch_utils.preparation.prepare_callbacks(cfg["model"]["trainer_params"])
already_defined = ["callbacks"]
trainer = torch_utils.preparation.prepare_trainer(**{k:cfg["model"]["trainer_params"][k] for k in cfg["model"]["trainer_params"] if k not in already_defined}, callbacks=callbacks)
loss = torch_utils.preparation.prepare_loss(cfg["model"]["loss"])
optimizer = torch_utils.preparation.prepare_optimizer(**cfg["model"]["optimizer"])

In [ ]:
model = torch_utils.process.create_model(main_module, loss, optimizer)

In [ ]:
torch_utils.process.train_model(trainer, model, loaders)